### Database
E scooter record: https://data.cityofchicago.org/Transportation/E-Scooter-Trips/2i5w-ykuw/about_data

Divvy record: https://data.cityofchicago.org/Transportation/Divvy-Trips/fg6s-gzvg/about_data


## Get ready for analysis

### Filter data from website
I chose person_type = 'BICYCLIST' from the Traffic Crashes – People database and vehicle_type = 'MOPED OR MOTORIZED BICYCLE' from the Traffic Crashes – Vehicles database to get the crash IDs for accidents involving electric bikes, bikes and mopeds for further analysis.

Because the e‑scooter record data is too large, I will focus my analysis on the year 2023.

## Data Preprocessing

In [22]:
import pandas as pd
import numpy as np
import doctest

### Load the data

Write the method for combining multiple csv

In [23]:
def load_multiple_csv(files,time_sig=False) -> pd.DataFrame:
    dfs=[]

    for file in files:
        df=pd.read_csv(file)
        if time_sig: # if we need to change the format of time
            df=time_change(df)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [24]:
%load_ext doctest
%doctest_mode

The doctest module is not an IPython extension.
Exception reporting mode: Context
Doctest mode is: OFF


In [25]:
def time_change(df:pd.DataFrame) : #the method of change the format of time
    """
    :param df:
    :return:
    """
    df["Start Time"] = pd.to_datetime(df["Start Time"],format="%m/%d/%Y %I:%M:%S %p")
    df["End Time"] = pd.to_datetime(df["End Time"],format="%m/%d/%Y %I:%M:%S %p")
    return df

Load the data of scooters had crash in 2024

In [35]:
crashes_scooter = pd.read_csv('Data/Traffic_Crashes_-V-Scooter1.csv')
crashes_scooter.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1973684,9d8272483c01c20bde2d269db47d217267846e5f6126fb...,12/21/2024 02:20:00 PM,1,DRIVER,NaN,1880498,NaN,UNKNOWN,MOTORIZED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971739,81b9bd1142a056a5dd0a1e6228e3abdd2444176800fda3...,12/18/2024 05:38:00 PM,1,DRIVER,NaN,1878624,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1971226,4fc7463c094e70a614198bd5d9555768da122382e70e93...,12/17/2024 10:56:00 PM,1,DRIVER,1.0,1878140,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1967911,92dce173be505a7846ca571fcee740e5e3a79d354bf614...,12/13/2024 12:58:00 PM,1,DRIVER,NaN,1876310,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1966169,f4ba62f428523478885a2c2576a9ac4d6c8b84aded732f...,12/10/2024 12:00:00 AM,2,PARKED,NaN,1873275,NaN,CHEVROLET,BLAZER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load the details of crashes

In [27]:
import glob
import os

In [28]:
cd=glob.glob(os.path.join("Data/Crashes-Crashes","*.csv"))
crashes_details=load_multiple_csv(cd,True)

In [29]:
crashes_details.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,LATITUDE,LONGITUDE
0,fde5e02103debbb95ac6b0e2391cd09fc30dbbf5a96d75...,05/02/2025 02:22:00 AM,10,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO INJURY / DRIVE AWAY,"$501 - $1,500",NOT APPLICABLE,41.865112,-87.623952
1,aa49b50898cebd8863b01a8b6e4da3cdcc3a2326803e3f...,05/01/2025 10:47:00 PM,15,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO INJURY / DRIVE AWAY,$500 OR LESS,DRIVING ON WRONG SIDE/WRONG WAY,41.764397,-87.583777
2,8c6dc50a4a5a5f4a73cdba0049c12c1630f4608014daa8...,05/01/2025 10:45:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",IMPROPER OVERTAKING/PASSING,41.869643,-87.645520
3,5a820f4840ac52672f2c15ba6246ddb6f4896191c1bad3...,05/01/2025 10:20:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO INJURY / DRIVE AWAY,"OVER $1,500",FAILING TO YIELD RIGHT-OF-WAY,41.750262,-87.663580
4,c3a270cd9bc718644ca132a4b3f356b15fd2e81fab9495...,05/01/2025 09:46:00 PM,25,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,INJURY AND / OR TOW DUE TO CRASH,"$501 - $1,500",UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN...,41.850762,-87.706295


combined the e-scooter record

In [47]:
sd=glob.glob(os.path.join("Data/ScooterTrips_Split","*.csv"))
scooter_loan_record=load_multiple_csv(sd,True)
scooter_loan_record.sort_values(by="End Time",ascending=True,inplace=True)

In [48]:
scooter_loan_record.head()

,Trip ID,Start Time,End Time,Trip Distance,Trip Duration,Start Centroid Latitude,Start Centroid Longitude,End Centroid Latitude,End Centroid Longitude
140290,83db5557-4329-45c5-b222-210bd9c7a986,2024-01-01 05:00:00,2024-01-01 05:00:00,1267,409,41.927261,-87.765502,41.924348,-87.734740
140282,77bc6c74-51cd-41f9-b117-6cd66713dc2e,2024-01-01 05:00:00,2024-01-01 05:00:00,1276,322,42.009623,-87.670167,42.001572,-87.695013
140283,81bfc84b-19f5-4455-b490-c304dab6dc91,2024-01-01 05:00:00,2024-01-01 05:00:00,588,107,42.009623,-87.670167,41.986712,-87.663417
140284,36022db7-81cf-471f-b261-7a0536ef7eb3,2024-01-01 05:00:00,2024-01-01 05:00:00,1769,394,41.741243,-87.551429,41.741243,-87.551429
140289,1007426f-4c84-4954-b328-1fbc06097b07,2024-01-01 05:00:00,2024-01-01 05:00:00,1354,400,41.927261,-87.765502,41.894101,-87.763112


### Combine the Data

Find the details of crashes from the file "Crashes-Crashes" with using CRASH_RECORD_ID, the CRASH_DATE will be later helpful in finding which loan record had a crash

In [42]:
scooter_crash_detail = crashes_details[crashes_details['CRASH_RECORD_ID'].isin(crashes_scooter['CRASH_RECORD_ID'])].copy()
scooter_crash_detail['CRASH_DATE']=pd.to_datetime(scooter_crash_detail['CRASH_DATE'],format="%m/%d/%Y %I:%M:%S %p")
scooter_crash_detail.sort_values(by="CRASH_DATE",ascending=True,inplace=True)
scooter_crash_detail.head

<bound method NDFrame.head of                                           CRASH_RECORD_ID          CRASH_DATE  \
144155  5ea616524de6e4a382b52972c780ea6b7d624212667e18... 2024-01-07 19:42:00   
137821  3729d925ef209747c0bfd1404ff53731a6bfd91c73416c... 2024-01-30 15:15:00   
135404  ade02228128dd40687cef57bc3effbe20086f09d20caf9... 2024-02-08 18:25:00   
133146  36dafcdcd10a0701a2579d9eebdd53466a95c326fd13f9... 2024-02-17 12:55:00   
132626  6cd5396550e83ead59c1b227d775d130403595327e5648... 2024-02-19 17:42:00   
...                                                   ...                 ...   
39806   f4ba62f428523478885a2c2576a9ac4d6c8b84aded732f... 2024-12-10 00:00:00   
38815   92dce173be505a7846ca571fcee740e5e3a79d354bf614... 2024-12-13 12:58:00   
37317   4fc7463c094e70a614198bd5d9555768da122382e70e93... 2024-12-17 22:56:00   
37075   81b9bd1142a056a5dd0a1e6228e3abdd2444176800fda3... 2024-12-18 17:38:00   
36177   9d8272483c01c20bde2d269db47d217267846e5f6126fb... 2024-12-21 14:20:00  

## In a crash, the scooter was loaned or owned

For now, we don't know which scooter crashes were loaned or owned by the people itself. we will first match time, then match distance.

### 5 min, 50 m

if a loan record end within 5 minutes of a crash happened, then we calculate if they are within 50 meters, if so, we consider this loan record happened a crash.

In [58]:
time_window = pd.Timedelta("5min")
max_dist = 50

### Distance Calculating

In [49]:
def distance(lat1,lon1,lat2,lon2):
    dy=(lat2-lat1)*111000
    dx=(lon2-lon1)*111320
    return np.hypot(dx,dy)

### Match the time and the distance

In [56]:
def crash_label(loans:pd.DataFrame,crash,verbose=False)-> str:
    window = ((loans['End Time'] >= crash['CRASH_DATE'] - time_window) &(loans['End Time'] <= crash['CRASH_DATE'] + time_window))
    candidates=loans.loc[window]
    if candidates.empty:
        return "owned"

    lat1,lon1=crash['LATITUDE'],crash['LONGITUDE']
    for lat2,lon2 in zip(candidates['End Centroid Latitude'],candidates['End Centroid Longitude']):
        if distance(lat1,lon1,lat2,lon2)<max_dist:
            return "rental"
    return "loan"

In [61]:
scooter_crash_detail["Mode"]=scooter_crash_detail.apply(
    lambda row: crash_label(scooter_loan_record, row),
    axis=1
)
scooter_crash_detail.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,LATITUDE,LONGITUDE,Mode
144155,5ea616524de6e4a382b52972c780ea6b7d624212667e18...,2024-01-07 19:42:00,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",FAILING TO YIELD RIGHT-OF-WAY,41.869805,-87.643776,owned
137821,3729d925ef209747c0bfd1404ff53731a6bfd91c73416c...,2024-01-30 15:15:00,25,CLEAR,DAYLIGHT,UNKNOWN,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,41.930921,-87.777026,owned
135404,ade02228128dd40687cef57bc3effbe20086f09d20caf9...,2024-02-08 18:25:00,25,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO INJURY / DRIVE AWAY,"$501 - $1,500",FAILING TO YIELD RIGHT-OF-WAY,41.927492,-87.680788,owned
133146,36dafcdcd10a0701a2579d9eebdd53466a95c326fd13f9...,2024-02-17 12:55:00,0,CLEAR,DAYLIGHT,DRY,INJURY AND / OR TOW DUE TO CRASH,$500 OR LESS,UNABLE TO DETERMINE,41.961552,-87.680107,loan
132626,6cd5396550e83ead59c1b227d775d130403595327e5648...,2024-02-19 17:42:00,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,INJURY AND / OR TOW DUE TO CRASH,$500 OR LESS,DRIVING ON WRONG SIDE/WRONG WAY,41.924560,-87.716846,owned


Now, we know every crash whether the scooter is rent.

In [63]:
print(scooter_crash_detail["Mode"].value_counts())

Mode
owned    116
loan      48
Name: count, dtype: int64


## Comparison